In [ ]:
fwd_sel_df

# cleaning the datasets to prepare for merging

In [6]:
def binarize_cols(df, nan_threshold=20):
    few_NaNs_cols = []
    for col in df.columns:
        if (df[col].nunique(dropna=False) == 2) or (df[col].nunique(dropna=True) == 2):
            unique_vals = df[col].unique()
            if (unique_vals[0] !=0 and unique_vals[1] !=0):
                print(f"col:{col}\n0 means:{unique_vals[0]}\n1 means: {unique_vals[1]}\n")
                df[col] = df[col].replace({unique_vals[0]: int(0), unique_vals[1]: int(1)})
            df[col].fillna(0, inplace=True) #now this will only make sense tho if we OHE first.    
    return df


def drop_columns(df, col_lst):
    for i in col_lst:
        if i in df.columns:
            df.drop(i, axis=1,inplace=True)
    return df

def lowercase_column(df, column_name):
    if column_name in df.columns:
        df[column_name] = df[column_name].astype(str).str.lower()
    return df

def count_candidates_by_district(df, district_column, new_column_name):
    district_counts = df.groupby(district_column).size().reset_index(name=new_column_name)
    df_merged = df.merge(district_counts, on=district_column)
    df_merged = df_merged[df_merged[new_column_name]>1]
    return df_merged

def move_column_to_front(df, col_lst):
    for column_name in col_lst:
        if column_name in df.columns:
            df = df[[column_name] + [col for col in df.columns if col != column_name]]
    return df

def get_info(df):
    print(f"COL VALUE TYPES \n{df.dtypes} \n\ndf shape:{df.shape}\n\nall the columns:\n{df.columns}")
    
    
def rename_column_if_exists(df, old_name, new_name):
    if old_name in df.columns:
        df.rename(columns={old_name: new_name}, inplace=True)
    return df
    
def replace_values_fill_na(df, column_name, replace_dict):
    if column_name in df.columns:
        df[column_name] = df[column_name].replace(replace_dict).fillna(0)
    return df
    
    
def get_ohe_cols(df, unique_limit=16, exclude = ['total_runners_house','total_runners']):
    #object_cols = df.select_dtypes(include=['object']).columns
    res = [col for col in df.columns if ((df[col].nunique(dropna=False) <= unique_limit) and (df[col].nunique(dropna=False)>2) and (col != 'total_runners') and (col != 'General Status') and (col !='Total Endorsements'))]
    return res


def convert_type(df, col_lst):
    for column_name in col_lst:
        if column_name in df.columns and df[column_name].dtype != 'int64':
            df[column_name] = df[column_name].astype(int)

            
def get_X_df(df, y_col):
    return df.drop(y_col, axis=1)

def zero_dropper(df):
    for col in df.columns:
        if ((df[col]==0).mean() >= 0.90):
            df = df.drop(col, axis=1)
    return df

def drop_VIF_col(X, threshold=2):
    while True:
        vif_info = pd.DataFrame()
        vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        vif_info['Column'] = X.columns
        vif_info = vif_info.sort_values('VIF', ascending=False)
        max_VIF = vif_info['VIF'].iloc[0]

        if max_VIF > threshold:
            to_drop = vif_info['Column'].iloc[0]
            print(f"Dropped: {to_drop} with VIF of {max_VIF}")
            X = X.drop(to_drop, axis=1)
        else:
            break

    return X  # Returning the modified DataFrame

def change_to_int(df, column):
    df[column] = df[column].astype(int)
    
    
def convert_to_int_if_possible(df, col_lst):
    for column_name in col_lst:
        temp_col = pd.to_numeric(df[column_name], errors='coerce')
        if temp_col.isna().sum() == 0:
            if all(temp_col.dropna() == temp_col.dropna().astype(int)):
                df[column_name] = temp_col.astype(int)
    return df

def aic(X, y):
    best_features = []
    best_aic = float('inf')

    for feature in X.columns:
        # Create a temporary DataFrame with the current set of best features plus the new feature
        X_temp = sm.add_constant(X[best_features + [feature]])

       
        model = sm.Logit(y, X_temp)  # Fit the logistic regression model
        result = model.fit()  # disp=0 suppresses the fit output

        # Check AIC and update if it is lower
        if result.aic < best_aic:
            best_aic = result.aic
            best_features.append(feature)

    # Fit the final model with the best features
    X_final = sm.add_constant(X[best_features])
    final_model = sm.Logit(y, X_final)
    final_result = final_model.fit()

    # You might want to return the final model, its summary, or AIC
    print(f"{best_features}")
    return final_result.summary(), final_result.aic

In [7]:
# #All subsets of size k
# def get_combinations(arr, k):
#     if k == 0:
#         return [[]]
#     if not arr: 
#         return []
#     return get_combinations(arr[1:], k) + [x + [arr[0]] for x in get_combinations(arr[1:], k-1)] 

# #Best Subsets Logisitc Regression using AIC

# def best_aic_per_k_function(X, y):
#     best_aic_per_k = {}
    
#     for k in range(1, 4): # Looping through all possible sizes of predictor subsets
    
#         best_aic_k = float("inf")    # Getting all combinations of predictors of size k without using itertools
#         best_model_k = None
#         best_variables_k = None
    
#     for variables in get_combinations(list(X.columns), k):
#         predictors = X[list(variables)]
#         predictors = sm.add_constant(predictors)  # Add a constant (intercept) to the model
        
#         # Fitting the model
#         model = sm.Logit(y, predictors).fit()  # Fit the logistic regression model
        
#         # Updating best AIC model for each k
#         if model.aic < best_aic_k:
#             best_aic_k = model.aic
#             best_model_k = model
#             best_variables_k = variables  # Storing variable names
    
    
#     best_aic_per_k[k] = (best_aic_k, best_model_k, best_variables_k) # Storing the best AIC and variable names for each k in the dictionary

#     for k, (aic, model, variables) in best_aic_per_k.items():  # Displaying the results
#         print(f"\nBest AIC value for k={k} variables: {aic}")
#         print(f"Variables in the model: {variables}")
#         print(f"Model parameters:\n{model.params}")

In [8]:
house = dem_candidates[dem_candidates['Office Type'] == 'Representative'] #filtering out non-house races
house = house[house['Race Type'] != 'Special']
house = rename_column_if_exists(house, 'Candidate', 'House Candidate')
house['dist_num'] = house['District'].str.extract('(\d+)$')
house['District Abbrev'] = house['State']+ '-' +house['dist_num']

In [101]:
qualities = ['Veteran?', 'LGBTQ?', 'STEM?', 'Race','Obama Alum?', 'Self-Funder?', 'Elected Official?'] #nulls in these columns mean no website for the candidate was found. 
# house[qualities] = house[qualities].replace({'No': 0, 'Yes': 1}).fillna(0)
# house[qualities] = house[qualities].replace({'White': 0, 'Nonwhite': 1}).fillna(0)
house = replace_values_fill_na(house,'General Status', {'None': 0, 'On the Ballot': 1})
house = house.reset_index(drop=True)

In [102]:
#of the resulting 22 rows, finding that 8 are NaNs due to the election not happenign yet. 
runoff_winners = ['Kendra Horn','Jason Nichols', 'Tim Gilpin', 'Mary Brannon']
house.loc[house['House Candidate'].isin(runoff_winners),'General Status'] = 'On the Ballot'
house.loc[house['House Candidate'].isin(runoff_winners), 'Primary Runoff Status'] = 'Advanced'
runoff_losers = ['Tom Guild', 'Clay Padgett','Amanda Douglas','Fred Gipson']
house.loc[house['House Candidate'].isin(runoff_losers),'General Status'] = 'None'

In [103]:
brookings = pd.read_csv('brookings.csv')

brookings = brookings[(brookings['Candidate.Party'].str.contains('Democrat')) & (brookings['Incumbent'].isnull())& (brookings['Primary.Outcome'].isin(['Winner','Loser']))]
brookings['Brookings Candidate'] = brookings['Candidate.First.Name'] + ' ' + brookings['Candidate.Last.Name']
brookings['District'] = brookings['Candidate.State'] + '-' + brookings['Candidate.District'].astype(str)
#brookings['Primary.Outcome'] = brookings['Primary.Outcome'].replace({'Loser': 0, 'Winner': 1})

replace_values_fill_na(brookings, 'Primary.Outcome', {'Loser': 0, 'Winner': 1})
brookings = drop_columns(brookings, ['Candidate.Party', 'Incumbent', 'Freshman.Member','Candidate.First.Name','Candidate.Last.Name','Republican',
                                     'Candidate.Website.URL', 'Primary.Runoff.Outcome', 'Party.Category.1','Candidate.State','Candidate.District',
                                     'Incumbency','Candidate.Gender','Democrat','Unnamed: 0'
                                    ])
brookings = move_column_to_front(brookings, ['Female','Brookings Candidate','District','Primary.Outcome'])
brookings = brookings.reset_index(drop=True)


In [75]:
house = drop_columns(house, ['dist_num', 'State', 'District', 'Office Type', 'Race Type',
       'Race Primary Election Date', 'Primary Status', 'Primary Runoff Status','Primary %', 'Won Primary'])

house = move_column_to_front(house, ['House Candidate','District Abbrev','General Status'])

In [76]:
house = count_candidates_by_district(house, 'District Abbrev', 'total_runners_house')
brookings = count_candidates_by_district(brookings, 'District', 'total_runners_brookings')

In [77]:
brookings = drop_columns(brookings, 'total_runners_brookings')

In [78]:
brookings = lowercase_column(brookings, 'Brookings Candidate')
house = lowercase_column(house, 'House Candidate')

In [79]:
house = house.reset_index(drop=True)
brookings = brookings.reset_index(drop=True)

# we can merge on 
- brookings['Primary.Outcome'] == house['General Status']
- brookings['District'] == house['District Abbrev']
- brookings['Candidate'] fuzzy match with house['Candidate'] with 60 percent match (this should be enough so long as the other two conditions are also matched)


Exporting this to two csvs to merge. Merging code in Merging Datasets

In [80]:
def fuzzy_join(brookings, house, threshold=50):
    matched_pairs = []

    for index1, row1 in brookings.iterrows():
        for index2, row2 in house.iterrows():
            if (row1['District'] == row2['District Abbrev'] and row1['Primary.Outcome'] == row2['General Status']):
                similarity_score = fuzz.ratio(row1['Brookings Candidate'], row2['House Candidate'])
                if similarity_score > threshold:
                    matched_pairs.append((index1, index2))
    matched_df = pd.DataFrame(columns=list(brookings.columns) + list(house.columns))
    for index1, index2 in matched_pairs:
        matched_row = pd.concat([brookings.iloc[[index1]].reset_index(drop=True), 
                                 house.iloc[[index2]].reset_index(drop=True)], axis=1)
        matched_df = matched_df.append(matched_row, ignore_index=True)

    return matched_df

In [81]:
output = fuzzy_join(brookings, house, threshold=70)

In [82]:
output = move_column_to_front(output,['Female','total_runners_house','Primary.Outcome','General Status','Brookings Candidate','House Candidate'])

In [83]:
change_to_int(output, 'Female')
change_to_int(output, 'General Status')
change_to_int(output, 'total_runners_house')

In [84]:
#output.to_csv(Path('joined_dfs.csv'))

In [155]:
output = drop_columns(output, ['District','Brookings Candidate', 'District Abbrev','House Candidate','Primary.Outcome'])
output = move_column_to_front(output, ['total_runners_house','total_runners_house','General Status'])

In [86]:
ohe_this = output.copy()